In [3]:
from efficient_apriori import apriori
import pandas as pd
import time

In [4]:
#读取csv文件
df = pd.read_csv('./ProjectB/订单表.csv',encoding = 'gbk')
#保留指定列
df = df[['客户ID','产品名称']]
#根据‘客户ID’列排序
df= df.sort_values(by = '客户ID', ascending = True)
#重命名列
df.rename(columns= {'客户ID': 'Transaction','产品名称':'Item'},inplace= True)
df

,Transaction,Item
24925,11000,挡泥板
32933,11000,车胎和内胎
25875,11000,山地自行车
34708,11000,山地自行车
25639,11000,头盔
...,...,...
37135,29480,水壶和水壶架
36875,29480,头盔
15205,29481,山地自行车
49016,29482,山地自行车


In [16]:
# 使用 efficient_apriori 里面的 apriori
def rule1():
    from efficient_apriori import apriori
    start= time.time()
# 获得一维series，并且将Transaction设为index
    order_series = df.set_index('Transaction')['Item']
    #转化数据到集合
    transactions =[]
    temp_index = 0
    for i,v in order_series.items():
        if i != temp_index:
            temp_set = set()
            temp_index = i
            temp_set.add(v)
            transactions.append(temp_set)
        else:
            temp_set.add(v)

    itemsets,rules = apriori(transactions,min_support = 0.02,min_confidence=0.5)
    print(f'频繁项集：{itemsets}')
    print(f'关联规则:{rules}')
    end = time.time()
    print(f'所用时间：{end-start}')

In [17]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
# 采用mlxtend.frequent_patterns工具包
def rule2():
	from mlxtend.frequent_patterns import apriori
	from mlxtend.frequent_patterns import association_rules
	pd.options.display.max_columns=100
	start = time.time()
	hot_encoded_df=df.groupby(['Transaction','Item'])['Item'].count().unstack().reset_index().fillna(0).set_index('Transaction')
	hot_encoded_df = hot_encoded_df.applymap(encode_units)
	frequent_itemsets = apriori(hot_encoded_df, min_support=0.02, use_colnames=True)
	rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.5)
	print("频繁项集：", frequent_itemsets)
	print("关联规则：", rules[ (rules['lift'] >= 2) & (rules['confidence'] >=0.5) ])
	#print(rules['confidence'])
	end = time.time()
	print("用时：", end-start)

In [18]:
rule1()
print('-'*98)
rule2()

频繁项集：{1: {('车胎和内胎',): 8490, ('头盔',): 5960, ('山地自行车',): 4089, ('骑行服',): 3192, ('旅行自行车',): 2143, ('挡泥板',): 2110, ('帽子',): 2132, ('公路自行车',): 6397, ('水壶和水壶架',): 4548, ('手套',): 1376, ('袜子',): 559, ('短裤',): 1019, ('水袋背包',): 719, ('清洁剂',): 875, ('背心',): 557}, 2: {('头盔', '山地自行车'): 1388, ('头盔', '挡泥板'): 742, ('头盔', '旅行自行车'): 1058, ('头盔', '车胎和内胎'): 2921, ('头盔', '骑行服'): 923, ('山地自行车', '挡泥板'): 898, ('山地自行车', '旅行自行车'): 1246, ('山地自行车', '车胎和内胎'): 1107, ('山地自行车', '骑行服'): 679, ('挡泥板', '车胎和内胎'): 723, ('挡泥板', '骑行服'): 493, ('旅行自行车', '车胎和内胎'): 529, ('旅行自行车', '骑行服'): 406, ('车胎和内胎', '骑行服'): 1150, ('公路自行车', '头盔'): 2051, ('公路自行车', '山地自行车'): 2053, ('公路自行车', '帽子'): 673, ('公路自行车', '挡泥板'): 493, ('公路自行车', '水壶和水壶架'): 1590, ('公路自行车', '骑行服'): 1154, ('头盔', '帽子'): 678, ('头盔', '水壶和水壶架'): 1453, ('山地自行车', '帽子'): 511, ('山地自行车', '水壶和水壶架'): 1246, ('帽子', '水壶和水壶架'): 746, ('帽子', '骑行服'): 689, ('挡泥板', '水壶和水壶架'): 524, ('水壶和水壶架', '骑行服'): 774, ('帽子', '旅行自行车'): 414, ('帽子', '车胎和内胎'): 760, ('旅行自行车', '水壶和水壶架'): 780, ('水壶和水壶架', '车胎和内胎'): 8